# 2 - Preparación de datos y herramientas

**Sumario**

1. Introducción
2. Información de entrada
3. Información de salida
4. Información de entrenamiento

## 2.1 - Introducción

El proceso de construcción de modelos de comportamiento basados en algoritmos de aprendizaje automático implica utilizar información que sea capaz de describir, de manera directa o indirecta, el comportamiento que queremos aprender. Es decir, si queremos construir un modelo de comportamiento que permita predecir, por ejemplo, el precio de una vivienda, deberíamos disponer de información descriptiva referida a las viviendas y a los precios a los que estas han sido vendidas.

Distinguimos tres tipos de información:

* **Información de entrada**. Información que será utilizada para construir el modelo mediante un algoritmo de aprendizaje. Normalmente, esta información ha de ser preprocesada, limpiada y transformada con el objetivo de prepararla para el proceso de entrenamiento.

* **Información de salida**. Información que se corresponde con el etiquetado (clase) de la información de entrada. Esta información es opcional, ya que en muchos caso no está disponible o o puede no utilizarse según la naturaleza del algoritmo de aprendizaje. Así, por ejemplo, la información de salida no es necesaria si usamos un algoritmo de aprendizaje no supervisado.

* **Información de entrenamiento**. Información que se corresponde con los tres elementos básicos del proceso de entrenamiento:
    * El **algoritmo de aprendizaje** que será utilizado 
    * El **algoritmo de optimización** que será utilizado con la funión de pérdida (*loss*)
    * **La función de pérdida (*loss*)** que será utilizada para medir la calidad del modelo durante el entrenamiento
    
<img src="images_2/tipos_de_informacion.png" width="600" data-align="center">

## 2.2 - Información de entrada

A la hora de definir la información de entrada, consideraremos que los datos se encuentran representados mediante **instancias formadas por atributos**.

Los **atributos** son las unidades básicas e indivisibles de información utilizadas para representar algún tipo de conocimiento específico sobre cada instancia. Este tipo de información representa una **característica básica y atómica** que intenta describir algún tipo de **rasgo sobre los datos** (color, tamaño, distancia, etc.). Los atributos suelen clasificarse en dos categorías en base a los valores que pueden adoptar:

* **Continuos**. Aquellos atributos que adoptan un valor fijo dentro de un **intervalo no finito perfectamente acotado** donde, dados dos valores observables, siempre existe un tercer valor intermedio que podría adoptar el atributo continuo. Por ejemplo, la temperatura media de una habitación
* **Discretos**. Aquellos atributos que adoptan su valor entre los elementos que forman un conjunto finito. Por ejemplo, los colores disponibles para pintar un vehiculo (rojo, azul, amarillo, etc.)

**Cada secuencia finita de atributos se agrupa en una instancia**, que es la estructura de información básica utilizada para representar cada uno de los ejemplos que conforman los conjuntos de datos que serán utilizados en el proceso de aprendizaje. **Las instancias de un mismo conjunto han de estar formadas por el mismo tipo de atributos**.

<img src="images_2/datos_entrada.png" width="600" data-align="center">

A la hora de utilizar conjuntos de datos es necesario analaizar la informacion contenida con el objetivo de asgurar dos propiedades fundamentales:
* **Calidad de la información.** ¿Contienen las instancias la información necesaria para aprender la tarea objetivo?
* **Cantidad de la información.** ¿Tenemos suficientes instancias para aprender la tarea objetivo?

In [8]:
import numpy as np
import pandas as pd

# automobile_nomissing = pd.read_csv("https://raw.githubusercontent.com/ferjorosa/telefonica-talentum-ai-10-2020/main/Data/automobile_nomissing.csv")
automobile = pd.read_csv("https://raw.githubusercontent.com/ferjorosa/telefonica-talentum-ai-10-2020/main/Data/automobile.csv")
automobile.head(5)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


### 2.2.1 - Calidad de la información

Como ya hemos mencionado, se trata de uno de los principales elementos que debemos tener en cuenta a la hora de construir un modelo. A la hora de utilizar un conjunto de datos, hay que analizar ciertos aspectos relacionados con ellos.

#### Valores atípicos
Los valores atípicos u outliers son valores que pueden adoptar los atributos y que resultan muy distantes con respecto a los valores que poseen el resto de instancias del conjunto. Estos valores pueden significar que se ha producido un error en la medición o que se ha detectado un caso inusual que no suele reproducirse.

Normalmente estos valores suelen ser eliminados, aunque hay que ser cuidadoso de no eliminar información relevante para el modelo. Algunos de los métodos más utilizados para la detección de estos valores son:

1. Métodos monovariables. Buscan valores atípicos de forma individual en cada atributo (con el fin de eliminarlos).
2. Métodos multivariables. Bsucan valores atípicos teniendo en consideración multiples atributos (con el fin de eliminarlos).
3. Método de error de Minkowski. A diferencia de los dos anteriores, no intenta eliminar los valores atípicos sino que los ranquea utilizando una variante del error cuadrático que se ve menos afectada por los valores atípicos. La principal diferencia es la introducción del parametro de Minkowski, que sustituye al cuadrado:

$$
\text{error_cuadratico} = \frac{\sum(\text{outputs} - \text{targets})^{2}}{\text{n_instancias}}
$$

$$
\text{error_miknowski} = \frac{\sum(\text{outputs} - \text{targets})^{\text{parametro_minkowski}}}{\text{n_instancias}}
$$

Un valor típico del parámetro de Minkowski es $1.5$. Por ejemplo, si un valor atípico tiene un error de 10, el error cuadrático para esa instancia es $10^{2}=100$, mientras que el error de Minkowski es $10^{1.5}=31.62$.

#### Uniformidad
Algunos ejemplos típicos de falta de uniformidad son:

* **Unidades de medición diferentes**. En muchos casos, a la hora de combinar información de diversas fuentes, nos podemos encontrar con que la información ha sido expresada en diferentes unidades métricas; así, por ejemplo, la altura de una persona se puede expresar en centímetros o en metros. Por tanto, es importante elegir el tipo de unidad de medida que se va a utilizar y aplicar las correspondientes transformaciones.

* **Falta de información**. En ciertos casos, algunos atributos carecen de valor dado que el sistema de extracción de información permite a los usuarios y a las usuarias no introducir un valor o bien existe un error en dicho proceso de extracción. Por tanto, es necesario decidir si esas instancias son eliminadas, si el atributo es ignorado, o si utilizamos algún método de estimación de valores missing.

* **Precisión en las mediciones**. En ciertos casos, el sensor con el que obtenemos las mediciones puede deteriorarse y con ello la calidad de los datos. Es importante tener esto en cuenta antes de aprender un modelo, no sea que consideremos datos no-reales por culpa del sensor en cuestión.

#### Ruido
El ruido es el conjunto de valores de un atributo específico que no aportan ningún tipo de información al proceso de aprendizaje y que pueden haber sido introducidos mediante un proceso de medición incorrecto (ruido de muestreo) o a causa de un error en la inserción de los datos.

Generalmente, el ruido presenta numerosas desventajas a la hora de construir un modelo, ya que tiende a aumentar la complejidad de los datos dificultando la ejecución de los algoritmos (i.e., le cuesta más esfuerzo al modelo encontrar los patrones "verdaderos").

<img src="images_2/ruido.png" width="500" data-align="center">

### 2.2.2 - Cantidad de la información

La falta de información puede hacer que el algoritmo de aprendizaje sea incapaz de construir un modelo que represente los patrones esperados. En base a esto, distinguimos dos tipos de problemas relacionados con la falta de información:

* **Falta de información global**, cuando tenemos muy poca información o todos los ejemplos son muy similares, dando lugar a un modelo que no es capaz de aprender.
* **Falta de información parcial**, cuando tenemos información no equilibrada entre las posibles salidas del modelo que queremos construir. Por ejemplo, si queremos construir un modelo que prediga los diferentes estados de la actividad física de una persona y tenemos para ello un conjunto de 5000 instancias distribuidos de la siguiente forma:
    1. 1700 ejemplos para *andar*
    2. 300 ejemplos para *correr ligeramente*
    3. 2800 ejemplos para *correr intensamente*
    4. 200 ejemplos para *quedarse quieto*
    
    Hay pocos ejemplos de *correr ligeramente* y *quedarse quieto*, lo cual puede producr que el modelo no sea capaz de predecir estos estados de forma confiable.

#### Aumentación

El proceso de aumentación de datos (*Data Augmentation*) consiste en **generar información artificial válida mediante la aplicación de sampleo o transformaciones simples** con el objetivo de aumentar el número de ejemplos disponibles en el conjunto de entrada. Esta técnica es muy útil cuando esos pequeños cambios no suponen una introducción de errores en el conjunto de datos.

Por ejemplo, si nuestro conjunto de datos es una imagen, el proceso de aumentación de datos podría generar nuevas aplicando pequeñas transformaciones sobre la imagen general.

### 2.2.3 - Preparación de datos

## 2.3 - Información de salida

## 2.4 - Información de entrenamiento